In [1]:
import findspark
import os
import gc
#findspark.init() 
SPARK_HOME='/opt/cloudera/parcels/CDH/lib/spark'
findspark.init(SPARK_HOME)

import time
import math
import copy
import csv
import json
import os
import codecs
import subprocess
import numpy as np
from pyspark import SQLContext
from pyspark.sql import Row
from pyspark.sql import functions as F
from pyspark.sql.functions import create_map,concat_ws,size,array_union,flatten,array_sort,coalesce,broadcast,collect_list, collect_set, udf, array_remove, log, lit, first, col, array, sort_array,split, explode, desc, asc, row_number,isnan, when, count
from pyspark.sql.types import *
import rtree
from pyspark.sql import Window
import geofeather
from pyspark.storagelevel import StorageLevel

from pyspark.sql import SparkSession
from pyspark import StorageLevel
import geopandas as gpd
import pandas as pd
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType, FloatType, ArrayType, MapType
from shapely.geometry import Point
from shapely.geometry import Polygon

from sedona.register import SedonaRegistrator
from sedona.core.SpatialRDD import SpatialRDD, PointRDD, CircleRDD, PolygonRDD, LineStringRDD
from sedona.core.enums import FileDataSplitter
from sedona.utils.adapter import Adapter
from sedona.core.spatialOperator import KNNQuery
from sedona.core.spatialOperator import JoinQuery
from sedona.core.spatialOperator import JoinQueryRaw
from sedona.core.spatialOperator import RangeQuery
from sedona.core.spatialOperator import RangeQueryRaw
from sedona.core.formatMapper.shapefileParser import ShapefileReader
from sedona.core.formatMapper import WkbReader
from sedona.core.formatMapper import WktReader
from sedona.core.formatMapper import GeoJsonReader
from sedona.sql.types import GeometryType
from sedona.core.enums import GridType
from sedona.core.SpatialRDD import RectangleRDD
from sedona.core.enums import IndexType
from sedona.core.geom.envelope import Envelope
from sedona.utils import SedonaKryoRegistrator, KryoSerializer

os.environ['PYSPARK_PYTHON'] = "./environment/bin/python"
os.environ['YARN_CONF_DIR'] = "/opt/cloudera/parcels/CDH/lib/spark/conf/yarn-conf"

In [ ]:
tin_file = input("Here is a programe to extract boundary relations, please input the absolute or relative path to your .ts file:")

# get the directory, basename of the input file
print("\n********************")
tin_directory = os.path.dirname(tin_file)
print("tin_directory: ", tin_directory)

directory_type = input("Is the data stored in hdfs(0) or Tri_data(1) or Tetra_data (2):") or "2"

if directory_type == '0':
    directory = 'hdfs_data'
elif directory_type == '1':
    directory = 'Tri_data'
else:
    directory = 'Tetra_data'
    
tin_basename = os.path.basename(tin_file) # input_vertices_2.off
print("tin_basename: ", tin_basename)

tin_filename = os.path.splitext(tin_basename)[0] # input_vertices_2
print("tin_filename: ", tin_filename)

tin_extension = os.path.splitext(tin_basename)[1] # .off
print("tin_extension: ", tin_extension)

print("\n********************")
print("This is a TIN file in \"%s\" format" % tin_extension)

filtra = 'yes'

# allocate the number of executors, the number of cores per executor, and the amount of memory per executor
Num_executor = input("spark.executor.instances:") or "64"
Num_core_per_executor = input("spark.executor.cores:") or "5"
Memory_executor = input("spark.executor.memory? Please end with 'g':") or "64g"
MemoryOverhead_executor = input("spark.executor.memoryOverhead? Please end with 'g':") or "8g"

Num_core_per_driver = input("spark.driver.cores:") or "5"
Memory_driver = input("spark.driver.memory? Please end with 'g':") or "64g"
MemoryOverhead_driver = input("spark.driver.memoryOverhead? Please end with 'g':") or "8g"

Num_shuffle_partitions = input("spark.sql.shuffle.partitions:") or "200"

In [3]:
'''
spark.executor.cores: # Number of concurrent tasks an executor can run, euqals to the number of cores to use on each executor
spark.executor.instances: # Number of executors for the spark application
spark.executor.memory: # Amount of memory to use for each executor that runs the task
spark.executor.memoryOverhead:
spark.driver.cores: # Number of cores to use for the driver process; the default number is 1
spark.driver.memory: # Amount of memory to use for the driver
spark.driver.maxResultSize: to define the maximum limit of the total size of the serialized result that a driver can store for each Spark collect action
spark.default.parallelism: # Default number of partitions in RDDs returned by transformations like join, reduceByKey, and parallelize when not set by user. It can be set as spark.executor.instances * spark.executor.cores * 2
spark.sql.shuffle.partitions: determine how many partitions are used when data is shuffled between nodes, e.g., joins or aggregations. usually 1~5 times of executor.instances * executor.cores
spark.memory.storageFraction: determines the fraction of the heap space that is allocated to caching RDDs and DataFrames in memory.
spark.kryoserializer.buffer.max: determine the maximum of data that can be serialized at once; this must be larger than any object we attempt to serialize
spark.rpc.message.maxSize: # Maximum message size (in MiB) to allow in "control plane" communication; generally only applies to map output size information sent between executors and the driver. To communicate between the nodes, Spark uses a protocol called RPC (Remote Procedure Call), which sends messages back and forth. The spark.rpc.message.maxSize parameter limits how big these messages can be. 
spark.sql.broadcastTimeout: Spark will wait for this amount of time before giving up on broadcasting a table. Broadcasting can take a long time if the table is large or if there is a shuffle operation before it.
spark.sql.autoBroadcastJoinThreshold: Spark will broadcast a table to all worker nodes when performing a join if its size is less than this value; -1 means disabling broadcasting
'''

date = time.strftime("%m,%d,%Y")
date_name = date.split(',')[0] + date.split(',')[1] + date.split(',')[2]

hour = time.strftime("%H,%M")
hour_name = hour.split(',')[0] + hour.split(',')[1]

spark_app_name = "Tetra_Spark_A1_BasedOn_D3_" + tin_filename + '_' + date_name + '_' + hour_name
print("spark_app_name:", spark_app_name)

spark = SparkSession \
.builder \
.appName(spark_app_name) \
.master('yarn') \
.config("spark.serializer", KryoSerializer.getName) \
.config('spark.jars','sedona-core-2.4_2.11-1.0.0-incubating.jar,sedona-sql-2.4_2.11-1.0.0-incubating.jar,sedona-python-adapter-2.4_2.11-1.0.0-incubating.jar,sedona-viz-2.4_2.11-1.0.0-incubating.jar,geotools-wrapper-geotools-24.0.jar,graphframes-0.8.0-spark2.4-s_2.11.jar') \
.config('spark.executor.cores', Num_core_per_executor) \
.config('spark.executor.instances', Num_executor) \
.config('spark.executor.memory', Memory_executor) \
.config('spark.executor.memoryOverhead', MemoryOverhead_executor) \
.config('spark.driver.cores', Num_core_per_driver) \
.config('spark.driver.memory', Memory_driver) \
.config('spark.driver.memoryOverhead', MemoryOverhead_driver) \
.config('spark.driver.maxResultSize', '0') \
.config('spark.dynamicAllocation.enabled', 'false') \
.config('spark.network.timeout', '10000001s') \
.config('spark.executor.heartbeatInterval', '10000000s') \
.config('spark.sql.shuffle.partitions', Num_shuffle_partitions) \
.config("spark.default.parallelism", '400') \
.config("spark.kryoserializer.buffer.max", "1024mb") \
.config('spark.rpc.message.maxSize', '256') \
.config("spark.sql.broadcastTimeout", "36000") \
.config("spark.sql.autoBroadcastJoinThreshold", "-1") \
.config("spark.sql.objectHashAggregate.sortBased.fallbackThreshold", "-1") \
.config('spark.yarn.dist.archives', '/path/to/py37.tar.gz#environment') \
.config("spark.python.profile", "true") \
.config("spark.eventLog.enabled", "true") \
.config("spark.eventLog.logStageExecutorMetrics", "true") \
.config("spark.blacklist.enabled", "false") \
.getOrCreate()

spark_app_name: Tetra_Spark_A1_BasedOn_D3_brain_11232025_2258


In [5]:
import graphframes
from graphframes import GraphFrame
from graphframes import *
from graphframes.lib import Pregel

### read input filesread input files

In [6]:
# read_tetra_order() is a function used to read tetrahedron from a csv file
def read_tetra_order(hdfs_tetra_origin):
    '''
    this function has two input parameters.
    filtra: 'yes' or 'no', yes means that the input csv file is ordered by default
    directory: a string denoting the directory to a tetrahedra file
    tin_filename: a string denoting the file name of a tetrahedra extension, e.g., 827_monviso
    '''
        
    schema_tetra_origin = StructType([ \
        StructField("tetra_order",IntegerType(),True), \
        StructField("r1",IntegerType(),True), \
        StructField("r2",IntegerType(),True), \
        StructField("r3",IntegerType(),True), \
        StructField("r4",IntegerType(),True), \
        StructField("r1_ele",FloatType(),True), \
        StructField("r2_ele",FloatType(),True), \
        StructField("r3_ele",FloatType(),True), \
        StructField("r4_ele",FloatType(),True) \
      ])

    df_tetra_origin = spark.read.format("csv") \
          .option("header", False) \
          .schema(schema_tetra_origin)\
          .load(hdfs_tetra_origin)
        
    return df_tetra_origin


# read tetrahedra
hdfs_tetra_origin = directory + "/" + tin_filename + '_filtra_tetra_sort.csv'

df_tetra_order = read_tetra_order(hdfs_tetra_origin)
df_tetra_order.printSchema()

root
 |-- tetra_order: integer (nullable = true)
 |-- r1: integer (nullable = true)
 |-- r2: integer (nullable = true)
 |-- r3: integer (nullable = true)
 |-- r4: integer (nullable = true)
 |-- r1_ele: float (nullable = true)
 |-- r2_ele: float (nullable = true)
 |-- r3_ele: float (nullable = true)
 |-- r4_ele: float (nullable = true)



### globally obtain VT relation

In [7]:
# function to get VT directly from DF_T
def get_VT(df_tetra_order):
    df_tetra_order = df_tetra_order.withColumn("tetra", sort_array(F.array("r1", "r2", "r3", "r4"), False))
    df_VT_init_1 = df_tetra_order.select("r1","tetra")
    df_VT_init_2 = df_tetra_order.select("r2","tetra")
    df_VT_init_3 = df_tetra_order.select("r3","tetra")
    df_VT_init_4 = df_tetra_order.select("r4","tetra")
    
    df_VT_union12 = df_VT_init_1.union(df_VT_init_2)
    df_VT_union123 = df_VT_union12.union(df_VT_init_3)
    df_VT_union1234 = df_VT_union123.union(df_VT_init_4)
    
    df_VT = df_VT_union1234.groupBy('r1').agg(collect_list('tetra').alias('VT'))
    df_VT = df_VT.withColumnRenamed('r1', 'Ver')
    
    return df_VT

In [8]:
df_VT = get_VT(df_tetra_order)

df_VT.printSchema()

root
 |-- Ver: integer (nullable = true)
 |-- VT: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)



### locally obtain VF, VE, EF, FT

In [9]:
from itertools import combinations
def get_Forman_rela_from_VT(Ver, VT):
    if VT:
        # store VF in multi_F, VE in multi_E, VV in pt_set
        multi_F = set()
        multi_E = set()
        # pt_set = set()
        
        FT = {}
        for tetra in VT:
            # VF
            # Generate combinations of 3 elements including Ver
            for comb in combinations(tetra, 3):
                if Ver in comb:
                    sorted_comb = sorted(comb, reverse=True)
                    multi_F.add(tuple(sorted_comb))
                    
            # VE
            for v in tetra:
                if v > Ver:
                    multi_E.add(tuple([v, Ver]))
                elif v < Ver:
                    multi_E.add(tuple([Ver, v]))
                # ignore if v == Ver
                
            # VV
            # pt_set.update(tetra)
            
            # FT
            if Ver in tetra:
                index_ver = tetra.index(Ver)
                if index_ver == 0: # the first vertex is the expected one
                    face_0 = [tetra[0], tetra[1], tetra[2]]
                    if tuple(face_0) in FT:
                        FT[tuple(face_0)].append(tetra)
                    else:
                        FT[tuple(face_0)] = [tetra]
                        
                    face_1 = [tetra[0], tetra[1], tetra[3]]
                    if tuple(face_1) in FT:
                        FT[tuple(face_1)].append(tetra)
                    else:
                        FT[tuple(face_1)] = [tetra]
                        
                    face_2 = [tetra[0], tetra[2], tetra[3]]
                    if tuple(face_2) in FT:
                        FT[tuple(face_2)].append(tetra)
                    else:
                        FT[tuple(face_2)] = [tetra]
                        
                elif index_ver == 1: # the second vertex is the expected one
                    face_0 = [tetra[0], tetra[1], tetra[2]]
                    if tuple(face_0) in FT:
                        FT[tuple(face_0)].append(tetra)
                    else:
                        FT[tuple(face_0)] = [tetra]
                        
                    face_1 = [tetra[0], tetra[1], tetra[3]]
                    if tuple(face_1) in FT:
                        FT[tuple(face_1)].append(tetra)
                    else:
                        FT[tuple(face_1)] = [tetra]
                        
                    face_2 = [tetra[1], tetra[2], tetra[3]]
                    if tuple(face_2) in FT:
                        FT[tuple(face_2)].append(tetra)
                    else:
                        FT[tuple(face_2)] = [tetra]
                        
                elif index_ver == 2: # the second vertex is the expected one
                    face_0 = [tetra[0], tetra[1], tetra[2]]
                    if tuple(face_0) in FT:
                        FT[tuple(face_0)].append(tetra)
                    else:
                        FT[tuple(face_0)] = [tetra]
                        
                    face_1 = [tetra[0], tetra[2], tetra[3]]
                    if tuple(face_1) in FT:
                        FT[tuple(face_1)].append(tetra)
                    else:
                        FT[tuple(face_1)] = [tetra]
                        
                    face_2 = [tetra[1], tetra[2], tetra[3]]
                    if tuple(face_2) in FT:
                        FT[tuple(face_2)].append(tetra)
                    else:
                        FT[tuple(face_2)] = [tetra]
                        
                elif index_ver == 3: # the second vertex is the expected one
                    face_0 = [tetra[0], tetra[1], tetra[3]]
                    if tuple(face_0) in FT:
                        FT[tuple(face_0)].append(tetra)
                    else:
                        FT[tuple(face_0)] = [tetra]
                        
                    face_1 = [tetra[0], tetra[2], tetra[3]]
                    if tuple(face_1) in FT:
                        FT[tuple(face_1)].append(tetra)
                    else:
                        FT[tuple(face_1)] = [tetra]
                        
                    face_2 = [tetra[1], tetra[2], tetra[3]]
                    if tuple(face_2) in FT:
                        FT[tuple(face_2)].append(tetra)
                    else:
                        FT[tuple(face_2)] = [tetra]
                    
        # VF
        multi_F_list = sorted(multi_F) # save more time when using list(multi_F)
        
        # VE
        multi_E_list = sorted(multi_E)
        
        # VV
        # pt_set.remove(Ver)
        # pt_list = sorted(pt_set)
        
        multi_EF = {}
        for face in multi_F_list:
            # Generate combinations of 3 elements including Ver
            for edge in combinations(face, 2):
                if Ver in edge:
                    sorted_edge = sorted(edge, reverse=True)
                    if tuple(sorted_edge) in multi_EF:
                        multi_EF[tuple(sorted_edge)].append(face)
                    else:
                        multi_EF[tuple(sorted_edge)] = [face]
            
        return multi_F_list, multi_E_list, multi_EF, FT
            
# StructType for Forman relations
union_Forman_rela_schema = StructType([
    StructField("VF", ArrayType(ArrayType(IntegerType())),True), 
    StructField("VE", ArrayType(ArrayType(IntegerType())),True),
    StructField("EF", MapType(ArrayType(IntegerType()), ArrayType(ArrayType(IntegerType()))),True),
    StructField('FT', MapType(ArrayType(IntegerType()), ArrayType(ArrayType(IntegerType()))),True)
])

get_Forman_rela_from_VT_udf = udf(get_Forman_rela_from_VT, union_Forman_rela_schema)

df_Forman_rela = df_VT.withColumn("Forman_rela", get_Forman_rela_from_VT_udf(df_VT.Ver, df_VT.VT))
# df_VF = df_VF.select("Ver", "VF")
df_Forman_rela.printSchema()

root
 |-- Ver: integer (nullable = true)
 |-- VT: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- Forman_rela: struct (nullable = true)
 |    |-- VF: array (nullable = true)
 |    |    |-- element: array (containsNull = true)
 |    |    |    |-- element: integer (containsNull = true)
 |    |-- VE: array (nullable = true)
 |    |    |-- element: array (containsNull = true)
 |    |    |    |-- element: integer (containsNull = true)
 |    |-- EF: map (nullable = true)
 |    |    |-- key: array
 |    |    |    |-- element: integer (containsNull = true)
 |    |    |-- value: array (valueContainsNull = true)
 |    |    |    |-- element: array (containsNull = true)
 |    |    |    |    |-- element: integer (containsNull = true)
 |    |-- FT: map (nullable = true)
 |    |    |-- key: array
 |    |    |    |-- element: integer (containsNull = true)
 |    |    |-- value: array (valueContainsNull = true)
 |    |    

In [10]:
t0 = time.time()

# num_row = df_Forman_rela.count()

t1 = time.time()
t_count_Forman_rela = t1 - t0
print("Time cost:", t_count_Forman_rela)
# print("number of rows:", num_row)

Time cost: 9.202957153320312e-05


### compute the edges, faces, and tetrahedra within a lower star

In [11]:
def get_LS_EFT(Ver, VT, VF, VE):
    LS_VT = []
    LS_VF = []    
    LS_VE = []   
    for tetra in VT:
        if Ver >= max(tetra):
            LS_VT.append(tetra)
    for face in VF:
        if Ver >= max(face):
            LS_VF.append(face)
    for edge in VE:
        if Ver >= max(edge):
            LS_VE.append(edge)
            
    return LS_VT, LS_VF, LS_VE

get_LS_EFT_schema = StructType([
    StructField("LS_tetra", ArrayType(ArrayType(IntegerType())),True), 
    StructField('LS_face',ArrayType(ArrayType(IntegerType())),True),
    StructField('LS_edge',ArrayType(ArrayType(IntegerType())),True)
])

get_LS_EFT_udf = udf(get_LS_EFT, get_LS_EFT_schema)

In [12]:
df_LS_EFT_init = df_Forman_rela.withColumn("LS_EFT", get_LS_EFT_udf(df_Forman_rela.Ver, df_Forman_rela.VT, df_Forman_rela.Forman_rela.VF, df_Forman_rela.Forman_rela.VE))

df_LS_EFT = df_LS_EFT_init.select('Ver', 'VT', col('LS_EFT.LS_tetra').alias('LS_VT'), col('LS_EFT.LS_face').alias('LS_VF'), col('LS_EFT.LS_edge').alias('LS_VE'))
df_LS_EFT.printSchema()

root
 |-- Ver: integer (nullable = true)
 |-- VT: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- LS_VT: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- LS_VF: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- LS_VE: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)



### compute Forman gradient

In [13]:
from collections import deque
def get_Forman(Ver, LS_tetra, LS_face, LS_edge):
    d = 3
    k = 0
    ST = deque() # ST stores k-simplices belonging to the lower star, the initial value is the vertex Ver
    ST.append(Ver)
    # LT = LS_face # LT stores the top simplices belonging to the lower star, here are faces
    crit_cell = [] # crit_cell will store critical simplices
    Forman_VE = [] # store VE pairs
    Forman_EF = [] # store EF pairs
    Forman_FT = [] # store FT pairs
    
    if (len(LS_edge)+len(LS_face)+len(LS_tetra)) == 0: # ver is a local minimum
        crit_cell.append([Ver])
        return crit_cell, Forman_VE, Forman_EF, Forman_FT
    
    def num_unpaired_edges(tri_inner, crit_cell_inner, Forman_vec_pair_VE_inner, Forman_vec_pair_EF_inner):
        # compute the number of unpaired edges in tri_inner
        edge0 = [tri_inner[0], tri_inner[1]]
        edge1 = [tri_inner[0], tri_inner[2]]
        
        num_unpaired_edge = 2
        paired_edge = [] # stores edges in crit_cell_inner, Forman_vec_pair_VE_inner, and Forman_vec_pair_EF_inner
        for crit_inner_temp in crit_cell_inner:
            if len(crit_inner_temp) == 2: # crit_inner_temp is a critical edge
                paired_edge.append(crit_inner_temp)
                
        for pair_vec in Forman_vec_pair_VE_inner: # pair_vec is a vector, it can be (ver, edge) or (edge, face)
            paired_edge.append(pair_vec[1])
        for pair_vec in Forman_vec_pair_EF_inner: # pair_vec is a vector, it can be (ver, edge) or (edge, face)
            paired_edge.append(pair_vec[0])
                
        potential_paired_edge = [edge0, edge1]
        if edge0 in paired_edge:
            num_unpaired_edge = num_unpaired_edge - 1
            potential_paired_edge.remove(edge0)
            
        if edge1 in paired_edge:
            num_unpaired_edge = num_unpaired_edge - 1
            potential_paired_edge.remove(edge1)
            
        return num_unpaired_edge, potential_paired_edge
    
    def get_EF_pair(ST_inner, crit_cell_inner, Forman_VE_inner, Forman_EF_inner):
        # this function is to get the EF pairs, E derives from CR while F derives from ST
        for i in range(len(ST_inner)):
            face = ST_inner[i]
            num_unpaired_edge, potential_paired_edge = num_unpaired_edges(face, crit_cell_inner, Forman_VE_inner, Forman_EF_inner)
            if num_unpaired_edge == 1:
                return [potential_paired_edge[0], face] # return the EF pair
            
    def num_unpaired_faces(tetra_inner, crit_cell_inner, Forman_vec_pair_EF_inner, Forman_vec_pair_FT_inner):
        # compute the number of unpaired faces in tetra_inner
        face0 = [tetra_inner[0], tetra_inner[1], tetra_inner[2]]
        face1 = [tetra_inner[0], tetra_inner[1], tetra_inner[3]]
        face2 = [tetra_inner[0], tetra_inner[2], tetra_inner[3]]
        
        num_unpaired_face = 3
        paired_face = [] # stores edges in crit_cell_inner, Forman_vec_pair_VE_inner, and Forman_vec_pair_EF_inner
        for crit_inner_temp in crit_cell_inner:
            if len(crit_inner_temp) == 3: # crit_inner_temp is a critical face
                paired_face.append(crit_inner_temp)
                
        for pair_vec in Forman_vec_pair_EF_inner: # pair_vec is a vector, it can be (edge, face) or (face, tetra)
            paired_face.append(pair_vec[1])
        for pair_vec in Forman_vec_pair_FT_inner: # pair_vec is a vector, it can be (edge, face) or (face, tetra)
            paired_face.append(pair_vec[0])
                
        potential_paired_face = [face0, face1, face2]
        if face0 in paired_face:
            num_unpaired_face = num_unpaired_face - 1
            potential_paired_face.remove(face0)
            
        if face1 in paired_face:
            num_unpaired_face = num_unpaired_face - 1
            potential_paired_face.remove(face1)
            
        if face2 in paired_face:
            num_unpaired_face = num_unpaired_face - 1
            potential_paired_face.remove(face2)
            
        return num_unpaired_face, potential_paired_face       
            
    def get_FT_pair(ST_inner, crit_cell_inner, Forman_EF_inner, Forman_FT_inner):
        # this function is to get the FT pairs, F derives from CR while T derives from ST
        for i in range(len(ST_inner)):
            tetra = ST_inner[i]
            num_unpaired_face, potential_paired_face = num_unpaired_faces(tetra, crit_cell_inner, Forman_EF_inner, Forman_FT_inner)
            if num_unpaired_face == 1:
                return [potential_paired_face[0], tetra] # return the FT pair
                
        
    
    while k <= d: # d == 3 for a tetrahedral mesh
        CR = ST # CR stores candidate critical simplices
        k = k + 1        
        if k == 1: # edges in the lower star
            ST = deque()
            for e in LS_edge:
                ST.append(e) # assign LS_edge to ST
        elif k == 2: # faces in the lower star
            ST = deque()
            for f in LS_face:
                ST.append(f) # assign LS_face to ST
        elif k == 3: # tetrahedra in the lower star
            ST = deque()
            for t in LS_tetra:
                ST.append(t) # assign LS_tetra to ST
        
        while len(CR) > 0:
            if k == 1: # get VE pairs
                pair_VE_V, pair_VE_E = CR.popleft(), ST.popleft()
                Forman_VE.append([pair_VE_V, pair_VE_E])
            elif k == 2: # get EF pairs
                pair_EF = get_EF_pair(ST, crit_cell, Forman_VE, Forman_EF)
                if pair_EF: # the EF pair exists
                    pair_EF_E = pair_EF[0]
                    pair_EF_F = pair_EF[1]
                    Forman_EF.append([pair_EF_E, pair_EF_F])
                    # remove the paired edges and faces
                    CR.remove(pair_EF_E)
                    ST.remove(pair_EF_F)
                else:
                    crit_cell.append(CR.popleft())
            elif k == 3: # get FT pairs
                pair_FT = get_FT_pair(ST, crit_cell, Forman_EF, Forman_FT)
                if pair_FT: # the FT pair exists
                    pair_FT_F = pair_FT[0]
                    pair_FT_T = pair_FT[1]
                    Forman_FT.append([pair_FT_F, pair_FT_T])
                    # remove the paired faces and tetra
                    CR.remove(pair_FT_F)
                    ST.remove(pair_FT_T)
                else:
                    crit_cell.append(CR.popleft())
                    
            elif k == 4:
                crit_cell.append(CR.popleft())
                
    return crit_cell, Forman_VE, Forman_EF, Forman_FT
                
# StructType for Crit_cell        
add_Crit_cell_schema = StructType([
    StructField('Crit_cell',ArrayType(IntegerType()),True)
])

# StructType for VE pairs        
add_VE_schema = StructType([
    StructField("VE_pair_V", IntegerType(),True), 
    StructField('VE_pair_E',ArrayType(IntegerType()),True)
])

# StructType for EF pairs        
add_multi_EF_schema = StructType([
    StructField("EF_pair_E", ArrayType(IntegerType()),True), 
    StructField('EF_pair_F',ArrayType(IntegerType()),True)
])

# StructType for FT pairs        
add_multi_FT_schema = StructType([
    StructField("FT_pair_F", ArrayType(IntegerType()),True), 
    StructField('FT_pair_T',ArrayType(IntegerType()),True)
])

# the whole StructType
get_Forman_schema = StructType([
    StructField("Crit_cell", ArrayType(ArrayType(IntegerType())), True),
    StructField("VE_pair", ArrayType(add_VE_schema), True),
    StructField("EF_pair", ArrayType(add_multi_EF_schema), True),
    StructField("FT_pair", ArrayType(add_multi_FT_schema), True)
])

# convert a function to an udf and determine the return type
# https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.sql.functions.udf.html
get_Forman_udf = udf(get_Forman, get_Forman_schema)

In [14]:
df_Forman = df_LS_EFT.withColumn("LS_Forman", get_Forman_udf(df_LS_EFT.Ver, df_LS_EFT.LS_VT, df_LS_EFT.LS_VF, df_LS_EFT.LS_VE))
df_Forman = df_Forman.select(col("Ver"), col("VT"), col("LS_Forman").alias("Forman"))
df_Forman.printSchema()
print("The number of partitions for df_Forman:", df_Forman.rdd.getNumPartitions())
# df_Forman.show()

root
 |-- Ver: integer (nullable = true)
 |-- VT: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- Forman: struct (nullable = true)
 |    |-- Crit_cell: array (nullable = true)
 |    |    |-- element: array (containsNull = true)
 |    |    |    |-- element: integer (containsNull = true)
 |    |-- VE_pair: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- VE_pair_V: integer (nullable = true)
 |    |    |    |-- VE_pair_E: array (nullable = true)
 |    |    |    |    |-- element: integer (containsNull = true)
 |    |-- EF_pair: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- EF_pair_E: array (nullable = true)
 |    |    |    |    |-- element: integer (containsNull = true)
 |    |    |    |-- EF_pair_F: array (nullable = true)
 |    |    |    |    |-- element: integer (containsNull = true)
 |    |-- FT_pair: array 

In [15]:
# time cost to get df_Forman

t_df_Forman_0 = time.time()
num_df_Forman= df_Forman.count()

t_df_Forman_1 = time.time()
t_df_Forman = t_df_Forman_1 - t_df_Forman_0
print("time cost of df_Forman:", t_df_Forman)

time cost of df_Forman: 17.596744060516357


In [16]:
# Forman_path = 'Tetra_data' + '/' + tin_filename + '_Forman_VT' + '.parquet'

# df_Forman = spark.read.parquet(Forman_path)
# df_Forman.printSchema()

## Methods to build Graph FT

#### prepare nodes of Graph FT

In [17]:
df_tetra_node = df_tetra_order.select(col("r1"), col("r2"), col("r3"), col("r4"))
df_tetra_node = df_tetra_node.withColumn("tetra_origin", F.array("r1", "r2", "r3", "r4"))
df_tetra_node = df_tetra_node.withColumn("tetra", sort_array("tetra_origin", False)).drop("tetra_origin") # sort_array("tri", False), False means descending order
df_tetra_node = df_tetra_node.select(col("tetra"))
df_tetra_node.printSchema()

root
 |-- tetra: array (nullable = false)
 |    |-- element: integer (containsNull = true)



In [18]:
# add a special triangle [-1, -1, -1] as node -1; this is useful to deal with boundary elements
# e.g., for a boundary edge, it has only one coboundary triangle; to guarantee a consistent encoding, each edge has two coboundary triangels, the other coboundary triangle for a boundary edge is [-1,-1,-1]
# columns_special = ['tetra', 'V2_node']
value_special = [(-1, -1, -1, -1)]
df_special_V2_node = spark.createDataFrame(value_special, ArrayType(IntegerType()))
df_special_V2_node.printSchema()
# df_special_V2_node.show()

root
 |-- value: array (nullable = true)
 |    |-- element: integer (containsNull = true)



In [19]:
df_graph_V2_node = df_tetra_node.union(df_special_V2_node)
df_graph_V2_node = df_graph_V2_node.withColumn("tetra_str", concat_ws(",", col("tetra")))
df_graph_V2_node_final = df_graph_V2_node.select(col("tetra_str").alias("id"))
df_graph_V2_node_final.printSchema()
# df_graph_V2_node_final.show()

root
 |-- id: string (nullable = false)



In [20]:
# time cost to get df_graph_V2_node_final

t_df_graph_V2_node_final_0 = time.time()
num_df_graph_V2_node_final = df_graph_V2_node_final.count()

t_df_graph_V2_node_final_1 = time.time()
t_df_graph_V2_node_final = t_df_graph_V2_node_final_1 - t_df_graph_V2_node_final_0
print("time cost of df_graph_V2_node_final:", t_df_graph_V2_node_final)

time cost of df_graph_V2_node_final: 4.264808416366577


#### prepare arcs of the Graph FT

In [21]:
df_Forman_FT_pair = df_Forman.select(col("Ver"), col("VT"), col("Forman.FT_pair").alias("FT_pair"))
df_Forman_FT_pair.printSchema()

root
 |-- Ver: integer (nullable = true)
 |-- VT: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- FT_pair: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- FT_pair_F: array (nullable = true)
 |    |    |    |-- element: integer (containsNull = true)
 |    |    |-- FT_pair_T: array (nullable = true)
 |    |    |    |-- element: integer (containsNull = true)



In [22]:
# retrieve co-boundary tetrahedron (FT relation)
def get_co_tetra(VT, FT_pair):
    if len(FT_pair) > 0:
        co_tetra_pair = []
        num_FT_pair = len(FT_pair) # obtain the number of FT_pairs
        def Co_tetra(f,t,vt):
            # f is sorted
            for t_temp in vt:
                # e0 = sorted([vt[0], vt[1]], reverse=True) # sort the triangle in descending order
                f0 = [t_temp[0], t_temp[1], t_temp[2]] # f0, f1, and f2 are in descending order by default
                f1 = [t_temp[0], t_temp[1], t_temp[3]]
                f2 = [t_temp[0], t_temp[2], t_temp[3]]
                f3 = [t_temp[1], t_temp[2], t_temp[3]]
                t = [int(t[0]), int(t[1]), int(t[2]), int(t[3])]
                t_temp = [int(t_temp[0]), int(t_temp[1]), int(t_temp[2]), int(t_temp[3])]
                # if e==e0 or e==e1 or e==e2 and t != t_temp:
                if f==f0 or f==f1 or f==f2 or f==f3:
                    if t != t_temp:
                        return t_temp # t_temp is a coboundary triangle of t in vt
                
        for i in range(num_FT_pair):
            FT_pair_f = FT_pair[i]['FT_pair_F'] # the i-th FT_pair
            FT_pair_t = FT_pair[i]['FT_pair_T']
            FT_pair_co_t = Co_tetra(FT_pair_f, FT_pair_t, VT)
            if FT_pair_co_t is None: # len(FT_pair_co_t) is empty, boundary tetrahedron
                FT_pair_co_t = [-1, -1, -1, -1]
            co_tetra_pair.append([FT_pair_co_t, FT_pair_t])
            
        return co_tetra_pair
    
# the whole StructType
get_co_tetra_schema = StructType([
    StructField("FT_pair_co_t", ArrayType(IntegerType()), True),
    StructField("FT_pair_t", ArrayType(IntegerType()), True)
])

# convert a function to an udf and determine the return type
# https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.sql.functions.udf.html
get_co_tetra_udf = udf(get_co_tetra, ArrayType(get_co_tetra_schema))

df_Forman_V2_edge = df_Forman_FT_pair.withColumn("FT_pair_inTetra", get_co_tetra_udf(df_Forman_FT_pair.VT, df_Forman_FT_pair.FT_pair))
df_Forman_V2_edge = df_Forman_V2_edge.select(col("Ver"), col("FT_pair_inTetra"))
df_Forman_V2_edge.printSchema()
# df_Forman_V2_edge.show()

root
 |-- Ver: integer (nullable = true)
 |-- FT_pair_inTetra: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- FT_pair_co_t: array (nullable = true)
 |    |    |    |-- element: integer (containsNull = true)
 |    |    |-- FT_pair_t: array (nullable = true)
 |    |    |    |-- element: integer (containsNull = true)



In [23]:
# explode df_Forman_V2_edge
df_Forman_V2_edge_split = df_Forman_V2_edge.select(explode(df_Forman_V2_edge.FT_pair_inTetra).alias("FT_pair_src_dst"))

df_Forman_V2_edge_split = df_Forman_V2_edge_split.select(col("FT_pair_src_dst.FT_pair_co_t").alias("arc_dst"), col("FT_pair_src_dst.FT_pair_t").alias("arc_src")).na.drop()
df_Forman_V2_edge_split.printSchema()
# df_Forman_V2_edge_split.show()

root
 |-- arc_dst: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- arc_src: array (nullable = true)
 |    |-- element: integer (containsNull = true)



In [24]:
df_Forman_V2_edge_split = df_Forman_V2_edge_split.withColumn("arc_dst_str", concat_ws(",", col("arc_dst")))
df_Forman_V2_edge_split = df_Forman_V2_edge_split.withColumn("arc_src_str", concat_ws(",", col("arc_src")))
df_graph_V2_edge_final = df_Forman_V2_edge_split.select(col("arc_src_str").alias("src"), col("arc_dst_str").alias("dst"))

df_graph_V2_edge_final.printSchema()
# df_graph_V2_node_final.show()

root
 |-- src: string (nullable = false)
 |-- dst: string (nullable = false)



In [25]:
# time cost to get df_graph_V2_edge_final

t_df_graph_V2_edge_final_0 = time.time()
num_df_graph_V2_edge_final= df_graph_V2_edge_final.count()

t_df_graph_V2_edge_final_1 = time.time()
t_df_graph_V2_edge_final = t_df_graph_V2_edge_final_1 - t_df_graph_V2_edge_final_0
print("time cost of df_graph_V2_edge_final:", t_df_graph_V2_edge_final)

time cost of df_graph_V2_edge_final: 27.223649263381958


#### read df_Forman from HDFS

In [26]:
# Forman_path = 'Tetra_data' + '/' + tin_filename + '_Forman_VT' + '.parquet'

# df_Forman = spark.read.parquet(Forman_path)
# df_Forman.printSchema()

### construct Graph_FT

In [27]:
graph_FT = GraphFrame(df_graph_V2_node_final, df_graph_V2_edge_final)

print("type of graph_V2:", type(graph_FT))

type of graph_V2: <class 'graphframes.graphframe.GraphFrame'>


In [28]:
# set a checkpoint directory to improve performance
# Checkpointing regularly helps recover from failures, clean shuffle files, shorten the
# lineage of the computation graph, and reduce the complexity of plan optimization.

spark.sparkContext.setCheckpointDir('checkpoints')

### Compute descending 3-manifolds (influence regions of maxima), which are the connected components of Graph_FT

In [29]:
t_con_0 = time.time()
# result_con consists of two columns, vertex id, component
result_con = graph_FT.connectedComponents()
result_con.printSchema()

t_con_1 = time.time()
t_con = t_con_1 - t_con_0
print("time cost of connected components:",t_con)

root
 |-- id: string (nullable = false)
 |-- component: long (nullable = true)

time cost of connected components: 105.97986674308777


In [30]:
print("time cost of df_Forman:", t_df_Forman)
print("time cost of t_df_graph_V2_node_final:", t_df_graph_V2_node_final)
print("time cost of t_df_graph_V2_edge_final:", t_df_graph_V2_edge_final)
print("time cost of connected components:", t_con)

print("total time cost:", t_df_Forman + t_df_graph_V2_node_final + t_df_graph_V2_edge_final + t_con)

time cost of df_Forman: 17.596744060516357
time cost of t_df_graph_V2_node_final: 4.264808416366577
time cost of t_df_graph_V2_edge_final: 27.223649263381958
time cost of connected components: 105.97986674308777
total time cost: 155.06506848335266


### Extract critical triangles, tetrahedra

In [31]:
df_crit = df_Forman.select(col("Ver"), col("VT"), col("Forman.Crit_cell").alias("Critical"))
print("number of partitions for df_crit:", df_crit.rdd.getNumPartitions())
df_crit.printSchema()
# df_crit.show()

number of partitions for df_crit: 200
root
 |-- Ver: integer (nullable = true)
 |-- VT: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- Critical: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)



In [32]:
# split the Critical column to obtain the critical vertices, edges, and triangles
def get_crit_VEFT(crit_cell):
    if len(crit_cell) < 1: # crit_cell is empty, there are no critical simplices within this lower star
        return
    else:
        # initialize the critical vertices, critical edges, and critical triangles
        crit_ver = None
        crit_edge = []
        crit_tri = []
        crit_tetra = []
        for icrit_cell_temp in crit_cell:
            if len(icrit_cell_temp) == 1: # icrit_cell_temp stores a critical vertex
                crit_ver = icrit_cell_temp[0]
            if len(icrit_cell_temp) == 2: # icrit_cell_temp stores critical edges
                crit_edge.append(icrit_cell_temp)
            if len(icrit_cell_temp) == 3: # icrit_cell_temp stores critical triangles
                crit_tri.append(icrit_cell_temp)
            if len(icrit_cell_temp) == 4: # icrit_cell_temp stores critical tetrahedra
                crit_tetra.append(icrit_cell_temp)
                
        if len(crit_edge) == 0:
            crit_edge = None
        if len(crit_tri) == 0:
            crit_tri = None
        if len(crit_tetra) == 0:
            crit_tetra = None
        return crit_ver, crit_edge, crit_tri, crit_tetra
        
# the whole StructType
get_crit_VEFT_schema = StructType([
    StructField("crit_ver", IntegerType(), True),
    StructField("crit_edge", ArrayType(ArrayType(IntegerType())), True),
    StructField("crit_tri", ArrayType(ArrayType(IntegerType())), True),
    StructField("crit_tetra", ArrayType(ArrayType(IntegerType())), True)
])

# convert a function to an udf and determine the return type
# https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.sql.functions.udf.html
get_crit_VEFT_udf = udf(get_crit_VEFT, get_crit_VEFT_schema)

df_crit_VEFT = df_crit.withColumn("crit_cell", get_crit_VEFT_udf(df_crit.Critical))
print("number of partitions for df_crit_VEFT:", df_crit_VEFT.rdd.getNumPartitions())
df_crit_VEFT.printSchema()
# df_crit_VEFT.show()

number of partitions for df_crit_VEFT: 200
root
 |-- Ver: integer (nullable = true)
 |-- VT: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- Critical: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- crit_cell: struct (nullable = true)
 |    |-- crit_ver: integer (nullable = true)
 |    |-- crit_edge: array (nullable = true)
 |    |    |-- element: array (containsNull = true)
 |    |    |    |-- element: integer (containsNull = true)
 |    |-- crit_tri: array (nullable = true)
 |    |    |-- element: array (containsNull = true)
 |    |    |    |-- element: integer (containsNull = true)
 |    |-- crit_tetra: array (nullable = true)
 |    |    |-- element: array (containsNull = true)
 |    |    |    |-- element: integer (containsNull = true)



#### critical triangles and tetrahedra

In [33]:
# to get critical triangles
df_crit_F_init = df_crit_VEFT.select(col("Ver"), col("VT"), col("crit_cell.crit_tri").alias("Max_tri_init")).na.drop()

# there could be multiple critical triangles within one lower star
df_crit_F = df_crit_F_init.select(col("Ver"), col("VT"), explode("Max_tri_init").alias("Max_tri")).na.drop()
df_crit_F.printSchema()
print("number of partitions for df_crit_F:", df_crit_F.rdd.getNumPartitions())
# df_crit_F.show()

root
 |-- Ver: integer (nullable = true)
 |-- VT: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- Max_tri: array (nullable = true)
 |    |-- element: integer (containsNull = true)

number of partitions for df_crit_F: 200


In [34]:
t_0 = time.time()

num_crit_F = df_crit_F.count()
print("num_crit_F: ", num_crit_F)

t_1 = time.time()
print("time for counting:", t_1 - t_0)

num_crit_F:  19715
time for counting: 13.86667513847351


In [35]:
# to get critical triangles
df_crit_T_init = df_crit_VEFT.select(col("Ver"), col("crit_cell.crit_tetra").alias("Max_tetra_init")).na.drop()

# there could be multiple critical triangles within one lower star
df_crit_T = df_crit_T_init.select(col("Ver"), explode("Max_tetra_init").alias("Max_tetra")).na.drop()
df_crit_T.printSchema()
print("number of partitions for df_crit_T:", df_crit_T.rdd.getNumPartitions())
# df_crit_T.show()

root
 |-- Ver: integer (nullable = true)
 |-- Max_tetra: array (nullable = true)
 |    |-- element: integer (containsNull = true)

number of partitions for df_crit_T: 200


In [36]:
t_0 = time.time()

num_crit_T = df_crit_T.count()
print("num_crit_T: ", num_crit_T)

t_1 = time.time()
print("time for counting:", t_1 - t_0)

num_crit_T:  9737
time for counting: 11.463854312896729


#### 1) left join result_con with 2-saddles tetra to identify the component that each saddles tetra belongs to

In [37]:
df_crit_F_VT = df_crit_F
df_crit_F_VT.printSchema()

root
 |-- Ver: integer (nullable = true)
 |-- VT: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- Max_tri: array (nullable = true)
 |    |-- element: integer (containsNull = true)



In [38]:
# retrieve co-boundary tetrahedra (FT relation)
def get_co_tetra_of_tri(Max_tri, VT):
    if len(VT) > 0:
        f = Max_tri
        tetra_12 = []
        num_VT = len(VT) # obtain the number of FT_pairs
        for t_temp in VT:
            f0 = [t_temp[0], t_temp[1], t_temp[2]] # e0, e1, and e2 are in descending order by default
            f1 = [t_temp[0], t_temp[1], t_temp[3]]
            f2 = [t_temp[0], t_temp[2], t_temp[3]]
            f3 = [t_temp[1], t_temp[2], t_temp[3]]
            if f==f0 or f==f1 or f==f2 or f==f3:
                tetra_12.append(t_temp)
            # if len(tri_12) == 2: # Maybe this triangle is at the boundary, so it has only one corresponding triangle in VT
                # break
        '''
        if len(tetra_12) == 1:
            t_special = [-1,-1,-1,-1]
            tetra_12.append(t_special)
        '''
            
        return tetra_12
    
# the whole StructType
get_saddle_edge_tri_schema = StructType([
    StructField("Saddle_CoTetra_t1", ArrayType(IntegerType()), True),
    StructField("Saddle_CoTetra_t2", ArrayType(IntegerType()), True)
])

# convert a function to an udf and determine the return type
# https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.sql.functions.udf.html
get_co_tetra_of_tri_udf = udf(get_co_tetra_of_tri, ArrayType(ArrayType(IntegerType())))

df_crit_F_tetra = df_crit_F_VT.withColumn("Saddle_CoTetra", get_co_tetra_of_tri_udf(df_crit_F_VT.Max_tri, df_crit_F_VT.VT))
df_crit_F_tetra = df_crit_F_tetra.select(col("Saddle_CoTetra"))
df_crit_F_tetra.printSchema()
# df_crit_E_tri.show()

root
 |-- Saddle_CoTetra: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)



In [39]:
# obtain the co-boundary tetrahedra of 2-saddles
df_crit_F2tetra = df_crit_F_tetra.select(explode("Saddle_CoTetra").alias("Saddle_CoTetra_split"))
# some nodes may belong to multiple saddles, we need to remove them
df_crit_F2tetra = df_crit_F2tetra.distinct()
df_crit_F2tetra.printSchema()

root
 |-- Saddle_CoTetra_split: array (nullable = true)
 |    |-- element: integer (containsNull = true)



In [40]:
# identify the saddle tetrahedra that each component have, we also need to convert tetraheda data type from array of integers to string at first
df_crit_F2tetra = df_crit_F2tetra.withColumn("Saddle_CoTetra_split_str", concat_ws(",", col("Saddle_CoTetra_split"))).drop(col("Saddle_CoTetra_split"))

result_con_saddle_FT = result_con.join(df_crit_F2tetra,result_con.id==df_crit_F2tetra.Saddle_CoTetra_split_str, "inner")

print("number of partitions for result_con_saddle_FT:", result_con_saddle_FT.rdd.getNumPartitions())
result_con_saddle_FT.printSchema()

number of partitions for result_con_saddle_FT: 200
root
 |-- id: string (nullable = false)
 |-- component: long (nullable = true)
 |-- Saddle_CoTetra_split_str: string (nullable = false)



In [41]:
# obtain all saddle tetrahedra for each connected component
unique_SdlTetra_per_con_FT = result_con_saddle_FT.groupBy('component').agg(collect_list('id').alias('multi_SdlTetra'))
unique_SdlTetra_per_con_FT.printSchema()

root
 |-- component: long (nullable = true)
 |-- multi_SdlTetra: array (nullable = true)
 |    |-- element: string (containsNull = true)



##### 2) left join df_graph_V2_edge_final with result_con to identify the connected component that each arc belongs to

In [42]:
# left join df_graph_V1_arc with result_con to identify which component that each arc belongs to
df_G_V2_arc_component = df_graph_V2_edge_final.join(result_con,df_graph_V2_edge_final.src==result_con.id, "left").drop('id')
df_G_V2_arc_component.printSchema()

root
 |-- src: string (nullable = false)
 |-- dst: string (nullable = false)
 |-- component: long (nullable = true)



##### 3) store each component in the same row of a DataFrame

In [43]:
# convert each arc to a dictionary
df_G_V2_arc_component_dict = df_G_V2_arc_component.withColumn("arcs", create_map(col('src'), col('dst'))).drop('src','dst')

# combine dictionaries belonging to an identical component to one list
df_G_V2_arc_component_dict_gpy = df_G_V2_arc_component_dict.groupBy('component').agg(collect_list('arcs').alias('subgraphs'))

# left join df_G_V1_arc_component_dict_gpy with unique_SdlPts_per_con to obtain SdlPts belonging to each component
df_G_V2_arc_component_dict_gpy = df_G_V2_arc_component_dict_gpy.join(unique_SdlTetra_per_con_FT,df_G_V2_arc_component_dict_gpy.component==unique_SdlTetra_per_con_FT.component, "left").select(df_G_V2_arc_component_dict_gpy.component, 'multi_SdlTetra', 'subgraphs')
df_G_V2_arc_component_dict_gpy.printSchema()

root
 |-- component: long (nullable = true)
 |-- multi_SdlTetra: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- subgraphs: array (nullable = true)
 |    |-- element: map (containsNull = true)
 |    |    |-- key: string
 |    |    |-- value: string (valueContainsNull = false)



#### get the ascending 1-manifold

In [44]:
import collections
def bfs_rdd(multi_SdlTetra, subgraphs):
    '''
    component: an integer or long data type
    multi_SdlTetra: an array of string or an array of integers
    subgraphs: an array of dictionary, where each key is a string (or integer) and the corresponding value is also a string (or integer)
    '''
    # Build the adjacency list from the list of edge dictionaries
    adj_list = collections.defaultdict(list)
    for edge_dict in subgraphs:
        # Assuming each dictionary has only one key-value pair representing an edge
        for source, destination in edge_dict.items():
            adj_list[source].append(destination)

    visited = set()
    # Use collections.deque for efficient O(1) appends and pops
    queue = collections.deque(multi_SdlTetra)

    while queue:
        node = queue.popleft() # Use popleft for BFS

        if node not in visited:
            visited.add(node)
            # Get neighbors from the adjacency list
            neighbors = adj_list.get(node, [])

            # Add each neighbor individually to the queue if not visited
            for neighbor in neighbors:
                if neighbor not in visited: # Optional: check here to avoid adding duplicates to queue
                     queue.append(neighbor)
                    
    return list(visited)

bfs_rdd_udf = udf(bfs_rdd, ArrayType(StringType()))

In [45]:
# Use mapPartitions to apply the bfs function to each partition of the RDD
# Each partition corresponds to a single graph that is processed by a single executor
t_V2_0 = time.time()

df_V2_result = df_G_V2_arc_component_dict_gpy.withColumn("one_manifold", bfs_rdd_udf(df_G_V2_arc_component_dict_gpy.multi_SdlTetra, df_G_V2_arc_component_dict_gpy.subgraphs))

num_df_V2_result = df_V2_result.count()
print("num_df_V2_result:", num_df_V2_result)

t_V2_1 = time.time()
print("Time cost to extract V2-Paths:", t_V2_1 - t_V2_0)

num_df_V2_result: 9738
Time cost to extract V2-Paths: 24.238641500473022
